In [4]:
import os
import shutil

# 입력 루트 (TuSimple 원본)
input_root = "/home1/hyunje0/.cache/kagglehub/datasets/manideep1108/tusimple/versions/5/TUSimple/train_set/clips"

# 출력 루트 (Test Set 대용)
output_root = "/home1/hyunje0/contrastive_test_set"
os.makedirs(output_root, exist_ok=True)

# 사용 클립 리스트
clip_dirs = ["0313-1", "0313-2", "0531", "0601"]

# counter = 0
# for clip in clip_dirs:
#     clip_path = os.path.join(input_root, clip)
#     for timestamp in os.listdir(clip_path):
#         timestamp_path = os.path.join(clip_path, timestamp)
#         target_image = os.path.join(timestamp_path, "20.jpg")

#         if os.path.isfile(target_image):
#             # 새로운 파일명: clip_timestamp.jpg
#             new_filename = f"{clip}_{timestamp}.jpg"
#             shutil.copy(target_image, os.path.join(output_root, new_filename))
#             counter += 1

counter = 0
for clip in clip_dirs:
    clip_path = os.path.join(input_root, clip)
    for timestamp in os.listdir(clip_path):
        timestamp_path = os.path.join(clip_path, timestamp)
        target_image = os.path.join(timestamp_path, "20.jpg")

        if os.path.isfile(target_image):
            counter += 1
        

print(f"✅ 총 {counter}개의 20.jpg 파일을 '{output_root}'에 복사했습니다.")

✅ 총 3626개의 20.jpg 파일을 '/home1/hyunje0/contrastive_test_set'에 복사했습니다.


In [2]:
!pip install --upgrade pip
!pip install tqdm pillow numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1


In [ ]:
import os, random
import numpy as np
from PIL import Image, ImageFilter
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torchvision import transforms
from torch import nn
import torchvision

# ─── 1. SimCLRModel 정의 (학습 모델과 동일하게) ─────────────────
class SimCLRModel(nn.Module):
    def __init__(self, backbone='resnet18', projection_dim=256):
        super().__init__()
        base = torchvision.models.resnet50(weights=None) if backbone == 'resnet18' else torchvision.models.resnet18(weights=None)
        self.encoder = nn.Sequential(*list(base.children())[:-1])  # remove fc
        feat_dim = base.fc.in_features
        self.projector = nn.Sequential(
            nn.Linear(feat_dim, 512),
            nn.ReLU(),
            nn.Linear(512, projection_dim)
        )

    def forward(self, x):
        h = self.encoder(x)            # shape: [B, feat_dim, 1, 1]
        h = h.view(h.size(0), -1)      # flatten: [B, feat_dim]
        z = self.projector(h)          # [B, projection_dim]
        return F.normalize(z, dim=1)   # L2 normalize across feature dim


# ─── 2. 노이즈 추가 함수 ─────────────────────────────
def add_light_noise_with_level(img: Image.Image, level: int) -> Image.Image:
    if level == 0:
        return img.copy()

    arr = np.array(img).astype(np.float32)
    radius = 5 * level
    intensity = 0.15 * level
    blur = img.filter(ImageFilter.GaussianBlur(radius))
    arr = np.clip(arr + intensity * np.array(blur), 0, 255)

    h, w, _ = arr.shape
    cx = int(w * random.uniform(0.3, 0.7))
    cy = int(h * random.uniform(0.0, 0.3))
    r = int(min(h, w) * 0.1 * level)
    y, x = np.ogrid[:h, :w]
    mask = np.exp(-((x - cx) ** 2 + (y - cy) ** 2) / (2 * (r / 2) ** 2))
    arr = np.clip(arr + (mask[..., None] * (0.1 * level)), 0, 255)

    gamma = 1.0 + 0.1 * level
    inv = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv) * 255 for i in range(256)]).astype(np.uint8)
    arr = table[arr.astype(np.uint8)]

    return Image.fromarray(arr.astype(np.uint8))


# ─── 3. transform 및 임베딩 함수 ─────────────────────
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

def get_embedding(model, img: Image.Image):
    model.eval()
    with torch.no_grad():
        tensor = transform(img).unsqueeze(0).cuda()   # [1, 3, 224, 224]
        feat = model(tensor)                          # [1, 256]
        return feat.squeeze(0)                        # → [256]


# ─── 4. Cosine Similarity 평가 함수 ────────────────────────────────
def evaluate_similarity(model_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 모델 로드
    model = SimCLRModel(backbone='resnet18', projection_dim=256).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    # 테스트 이미지 5장
    test_root = "/home1/hyunje0/contrastive_test_set"
    image_list = sorted([f for f in os.listdir(test_root) if f.endswith(".jpg")])[:5]

    # 비교할 레벨 쌍
    compare_pairs = [(0, 1), (0, 3), (0, 5), (0, 8), (3, 7)]

    print("Cosine Similarity between noise levels using SimCLR model:\n")

    for fn in tqdm(image_list):
        img = Image.open(os.path.join(test_root, fn)).convert("RGB")
        level_feats = {}

        for lvl in range(9):  # 0~8
            noisy = add_light_noise_with_level(img, level=lvl)
            feat = get_embedding(model, noisy)
            level_feats[lvl] = feat

        print(f"[{fn}]")
        for a, b in compare_pairs:
            sim = F.cosine_similarity(level_feats[a], level_feats[b], dim=0).item()
            print(f"  - level {a}-{b}: cosine similarity = {sim:.4f}")
        print()


# ─── 5. 실행 ───────────────────────────────
evaluate_similarity("Models/best_model.pth")

In [5]:
def show_selected_images():
    import matplotlib.pyplot as plt

    test_root = "/home1/hyunje0/contrastive_test_set"
    image_list = sorted([f for f in os.listdir(test_root) if f.endswith(".jpg")])[:5]

    plt.figure(figsize=(15, 5))
    for i, fname in enumerate(image_list):
        img = Image.open(os.path.join(test_root, fname)).convert("RGB")
        plt.subplot(1, 5, i + 1)
        plt.imshow(img)
    
        plt.title(fname, fontsize=8)
        plt.axis("off")
    plt.suptitle("Selected TuSimple Test Images (Top 5)", fontsize=14)
    plt.tight_layout()
    plt.show()

show_selected_images()

NameError: name 'Image' is not defined

<Figure size 1500x500 with 0 Axes>

In [ ]:
import matplotlib.pyplot as plt
import os, random
from PIL import Image, ImageFilter
def show_augmented_images():
    test_root = "/home1/hyunje0/contrastive_test_set"
    image_list = random.sample(
    [f for f in os.listdir(test_root) if f.endswith(".jpg")],
    k=5)

    for fn in image_list:
        img = Image.open(os.path.join(test_root, fn)).convert("RGB")
        plt.figure(figsize=(18, 2.5))
        for lvl in range(9):  # level 0~8
            aug = add_light_noise_with_level(img, level=lvl)
            plt.subplot(1, 9, lvl+1)
            plt.imshow(aug)
            plt.title(f"lv {lvl}", fontsize=8)
            plt.axis("off")
        plt.suptitle(f"{fn} - Noise Levels 0~8", fontsize=12)
        plt.tight_layout()
        plt.show()

# 실행
show_augmented_images()

In [ ]:
# ─── 환경 설정 ──────────────────────────────────────────────────────────────
import os, random
import numpy as np
from PIL import Image, ImageFilter
import torch
import torch.nn.functional as F
from torch import nn
from torchvision import transforms
import torchvision
import matplotlib.pyplot as plt
from tqdm import tqdm

# ─── 디바이스 설정 ──────────────────────────────────────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# ─── 모델 정의 ──────────────────────────────────────────────────────────────
class SimCLR_Eval(nn.Module):
    def __init__(self, proj_dim=128):
        super().__init__()
        resnet = torchvision.models.resnet18(weights=None)
        self.encoder = nn.Sequential(*list(resnet.children())[:-2], nn.AdaptiveAvgPool2d((1, 1)))
        self.projector = nn.Sequential(
            nn.Linear(512, 512, bias=False), nn.BatchNorm1d(512), nn.ReLU(inplace=True),
            nn.Linear(512, 512, bias=False), nn.BatchNorm1d(512), nn.ReLU(inplace=True),
            nn.Linear(512, proj_dim, bias=False), nn.BatchNorm1d(proj_dim, affine=False)
        )

    def forward(self, x):
        h = self.encoder(x).flatten(1)
        z = F.normalize(self.projector(h), dim=1)
        return z

# ─── 모델 로드 ──────────────────────────────────────────────────────────────
model = SimCLR_Eval(proj_dim=128).to(device)
ckpt = torch.load("best_model.pth", map_location=device)
model.load_state_dict(ckpt, strict=True)
model.eval()

# ─── 노이즈 추가 함수 ───────────────────────────────────────────────────────
def add_light_noise_with_level(img: Image.Image, level: int) -> Image.Image:
    if level == 0:
        return img.copy()

    arr = np.array(img).astype(np.float32)
    radius = 5 * level
    intensity = 0.15 * level
    blur = img.filter(ImageFilter.GaussianBlur(radius))
    arr = np.clip(arr + intensity * np.array(blur), 0, 255)

    h, w, _ = arr.shape
    cx = int(w * random.uniform(0.3, 0.7))
    cy = int(h * random.uniform(0.0, 0.3))
    r = int(min(h, w) * 0.1 * level)
    y, x = np.ogrid[:h, :w]
    mask = np.exp(-((x - cx) ** 2 + (y - cy) ** 2) / (2 * (r / 2) ** 2))
    arr = np.clip(arr + (mask[..., None] * (0.1 * level)), 0, 255)

    gamma = 1.0 + 0.1 * level
    inv = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv) * 255 for i in range(256)]).astype(np.uint8)
    arr = table[arr.astype(np.uint8)]

    return Image.fromarray(arr.astype(np.uint8))

# ─── 전처리 및 임베딩 추출 ───────────────────────────────────────────────────
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5] * 3, [0.5] * 3)
])

def get_embedding(img: Image.Image):
    with torch.no_grad():
        x = transform(img).unsqueeze(0).to(device)
        f = model(x).squeeze()
        return f / f.norm()

# ─── 테스트 이미지 유사도 계산 ───────────────────────────────────────────────
base_path = "/home1/hyunje0/contrastive_test_set"
levels = [0, 3, 5, 8]

image_list = random.sample(
    [f for f in os.listdir(base_path) if f.endswith(".jpg")],
    k=5
)

for fname in tqdm(image_list, desc="Cosine Similarity on Test Images"):
    img = Image.open(os.path.join(base_path, fname)).convert("RGB")
    features = {}
    visuals = {}

    for lv in levels:
        aug = add_light_noise_with_level(img, lv)
        feat = get_embedding(aug)
        features[lv] = feat
        visuals[lv] = aug.copy()

    print(f"\n📄 {fname}")
    for lv in levels[1:]:
        sim = F.cosine_similarity(features[0], features[lv], dim=0).item()
        print(f"  - lv0 vs lv{lv}: cosine similarity = {sim:.4f}")

    fig, axes = plt.subplots(1, 4, figsize=(14, 3))
    for i, lv in enumerate(levels):
        axes[i].imshow(visuals[lv])
        axes[i].axis("off")
        axes[i].set_title(f"lv{lv}")
    plt.suptitle(f"{fname} - Noise Levels 0/3/5/8", fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
import os, random
import numpy as np
from PIL import Image, ImageFilter
import torch
import torch.nn.functional as F
from torch import nn
from torchvision import transforms
import torchvision
import matplotlib.pyplot as plt
from tqdm import tqdm

# ───────────────────────────────
# 1. 모델 정의 및 로딩
# ───────────────────────────────
class SimCLR_Eval(nn.Module):
    def __init__(self, proj_dim=128):
        super().__init__()
        resnet = torchvision.models.resnet18(weights=None)
        self.encoder = nn.Sequential(*list(resnet.children())[:-2], nn.AdaptiveAvgPool2d((1, 1)))
        self.projector = nn.Sequential(
            nn.Linear(512, 512, bias=False), nn.BatchNorm1d(512), nn.ReLU(inplace=True),
            nn.Linear(512, 512, bias=False), nn.BatchNorm1d(512), nn.ReLU(inplace=True),
            nn.Linear(512, proj_dim, bias=False), nn.BatchNorm1d(proj_dim, affine=False)
        )

    def forward(self, x):
        h = self.encoder(x).flatten(1)
        z = F.normalize(self.projector(h), dim=1)
        return z

# ✅ 모델 로드 (encoder + projector 모두 로딩)
model = SimCLR_Eval(proj_dim=128)
ckpt = torch.load("Models/best_model.pth", map_location="cpu")
model.load_state_dict(ckpt, strict=True)
model.eval()

# ───────────────────────────────
# 2. 노이즈 함수 정의
# ───────────────────────────────
def add_light_noise_with_level(img: Image.Image, level: int) -> Image.Image:
    if level == 0:
        return img.copy()

    arr = np.array(img).astype(np.float32)
    radius = 5 * level
    intensity = 0.15 * level
    blur = img.filter(ImageFilter.GaussianBlur(radius))
    arr = np.clip(arr + intensity * np.array(blur), 0, 255)

    h, w, _ = arr.shape
    cx = int(w * random.uniform(0.3, 0.7))
    cy = int(h * random.uniform(0.0, 0.3))
    r = int(min(h, w) * 0.1 * level)
    y, x = np.ogrid[:h, :w]
    mask = np.exp(-((x - cx) ** 2 + (y - cy) ** 2) / (2 * (r / 2) ** 2))
    arr = np.clip(arr + (mask[..., None] * (0.1 * level)), 0, 255)

    gamma = 1.0 + 0.1 * level
    inv = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv) * 255 for i in range(256)]).astype(np.uint8)
    arr = table[arr.astype(np.uint8)]

    return Image.fromarray(arr.astype(np.uint8))

# ───────────────────────────────
# 3. 전처리 및 임베딩 추출 함수
# ───────────────────────────────
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5] * 3, [0.5] * 3)
])

def get_embedding(img: Image.Image):
    with torch.no_grad():
        x = transform(img).unsqueeze(0)
        f = model(x).squeeze()
        return f / f.norm()

# ───────────────────────────────
# 4. 이미지 로딩 및 cosine similarity 계산
# ───────────────────────────────
base_path = "/home1/hyunje0/contrastive_test_set"
levels = [0, 3, 5, 8]

image_list = random.sample(
    [f for f in os.listdir(base_path) if f.endswith(".jpg")],
    k=5
)

for fname in tqdm(image_list, desc="Cosine Similarity on Test Images"):
    img = Image.open(os.path.join(base_path, fname)).convert("RGB")
    features = {}
    visuals = {}

    for lv in levels:
        aug = add_light_noise_with_level(img, lv)
        feat = get_embedding(aug)
        features[lv] = feat
        visuals[lv] = aug.copy()

    # Cosine Similarity 출력
    print(f"\n📄 {fname}")
    for lv in levels[1:]:
        sim = F.cosine_similarity(features[0], features[lv], dim=0).item()
        print(f"  - lv0 vs lv{lv}: cosine similarity = {sim:.4f}")

    # 시각화
    fig, axes = plt.subplots(1, 4, figsize=(14, 3))
    for i, lv in enumerate(levels):
        axes[i].imshow(visuals[lv])
        axes[i].axis("off")
        axes[i].set_title(f"lv{lv}")
    plt.suptitle(f"{fname} - Noise Levels 0/3/5/8", fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
import os, random
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchvision

# ───────────────────────────────
# 0. Config & Device
# ───────────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# data_dir = "/home1/hyunje0/.cache/kagglehub/datasets/solesensei/solesensei_bdd100k/versions/2/aug_8levels/img"
data_dir = "/home1/hyunje0/contrastive_test_set"
model_path = "Models/best_model.pth"

# ───────────────────────────────
# 1. 모델 정의 (encoder + projector)
# ───────────────────────────────
def resnet18_backbone():
    m = torchvision.models.resnet18(weights=None)
    return nn.Sequential(*list(m.children())[:-2], nn.AdaptiveAvgPool2d((1, 1)))

class SimCLR(nn.Module):
    def __init__(self, proj_dim=128):
        super().__init__()
        self.encoder = resnet18_backbone()
        self.projector = nn.Sequential(
            nn.Linear(512, 512, bias=False), nn.BatchNorm1d(512), nn.ReLU(inplace=True),
            nn.Linear(512, 512, bias=False), nn.BatchNorm1d(512), nn.ReLU(inplace=True),
            nn.Linear(512, proj_dim, bias=False), nn.BatchNorm1d(proj_dim, affine=False)
        )

    def forward(self, x):
        h = self.encoder(x).flatten(1)
        z = F.normalize(self.projector(h), dim=1)
        return z

# ───────────────────────────────
# 2. 모델 로딩
# ───────────────────────────────
model = SimCLR(proj_dim=128).to(device)
ckpt = torch.load(model_path, map_location=device)
model.load_state_dict(ckpt, strict=True)
model.eval()

# ───────────────────────────────
# 3. Transform + Embedding 함수
# ───────────────────────────────
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5] * 3, [0.5] * 3)
])

def get_embedding(img: Image.Image):
    with torch.no_grad():
        x = transform(img).unsqueeze(0).to(device)
        z = model(x).squeeze()
        return z / z.norm()

# ───────────────────────────────
# 4. Collapse 여부 검증
# ───────────────────────────────
image_list = [f for f in os.listdir(data_dir) if f.endswith((".jpg", ".png"))]
sampled = random.sample(image_list, min(100, len(image_list)))  # 최대 100장

feats = []
for fname in sampled:
    img = Image.open(os.path.join(data_dir, fname)).convert("RGB")
    feats.append(get_embedding(img))

all_feats = torch.stack(feats)  # [N, D]
mean_std = all_feats.std(dim=0).mean().item()

print(f"\n📊 평균 Feature 표준편차 (차원별 std의 평균): {mean_std:.6f}")

# ───────────────────────────────
# 5. 해석
# ───────────────────────────────
if mean_std < 0.01:
    print("⚠️ Feature collapse 의심: 대부분 임베딩이 동일합니다.")
elif mean_std < 0.05:
    print("⚠️ Feature 다양성이 부족할 수 있음 (약한 collapse 가능)")
else:
    print("✅ Feature 다양성이 충분합니다.")

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"✅ CUDA 사용 가능 (GPU 사용 중) — GPU 이름: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("⚠️ CUDA 사용 불가 (CPU 사용 중)")

In [ ]:
# ----------------------구분선---------------------------------

In [1]:
all_imgs = [f for f in os.listdir(base_path) if f.endswith(".jpg")]
base_imgs = random.sample(all_imgs, k=100)

NameError: name 'os' is not defined

In [ ]:
from collections import defaultdict

# 1. 데이터 준비
all_imgs = [f for f in os.listdir(base_path) if f.endswith(".jpg")]
base_imgs = random.sample(all_imgs, k=100)

same_sims = defaultdict(list)
diff_sims = []

for fname in tqdm(base_imgs, desc="Experiment loops"):
    img = Image.open(os.path.join(base_path, fname)).convert("RGB")
    emb0 = get_embedding(img)

    # 2. 동일 이미지 vs 노이즈
    for lv in [3,5,8]:
        aug = add_light_noise_with_level(img, lv)
        embn = get_embedding(aug)
        sim = F.cosine_similarity(emb0, embn, dim=0).item()
        same_sims[lv].append(sim)

    # 3. 서로 다른 이미지 간 유사도
    other = random.choice([x for x in all_imgs if x != fname])
    img2 = Image.open(os.path.join(base_path, other)).convert("RGB")
    emb2 = get_embedding(img2)
    sim_diff = F.cosine_similarity(emb0, emb2, dim=0).item()
    diff_sims.append(sim_diff)

In [ ]:
import matplotlib.pyplot as plt

# same-image 분포
for lv, sims in same_sims.items():
    plt.hist(sims, bins=20, alpha=0.5, label=f"same lv{lv}")
# diff-image 분포
plt.hist(diff_sims, bins=20, alpha=0.5, label="diff lv0")
plt.legend()
plt.xlabel("Cosine Similarity")
plt.ylabel("Frequency")
plt.title("Same vs Diff Image Similarity Distributions")
plt.show()